In [1]:
import torch
import torch.nn.functional as F
import torchsde

from torchvision import datasets, transforms

import math
import numpy as np
import pandas as pd
from tqdm import tqdm

from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import functorch

import matplotlib.pyplot as plt

import cfollmer.functional as functional
from cfollmer.objectives import relative_entropy_control_cost
from cfollmer.drifts import SimpleForwardNet, SimpleForwardNetBN, ResNetScoreNetwork
from cfollmer.sampler_utils import FollmerSDE

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class DNN(torch.nn.Module):
    
    def __init__(self, input_dim=1, output_dim=1):
        super(DNN, self).__init__()
        
        self.output_dim = output_dim
        self.input_dim = input_dim
        
        self.nn = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, output_dim)
        )
        
    def forward(self, x):
        return self.nn(x)

    
class LinModel(torch.nn.Module):
    
    def __init__(self, input_dim=1, output_dim=1):
        super(LinModel, self).__init__()
        
        self.output_dim = output_dim
        self.input_dim = input_dim
        
        self.nn = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 1),
        )
        
    def forward(self, x):
        return self.nn(x)

In [4]:
def train(X_train, y_train, gamma, dim, n_steps, data_batch_size, param_batch_size, dt=0.05, stl=False, size_list=None):
    
    sde = FollmerSDE(gamma, SimpleForwardNetBN(input_dim=dim, width=300)).to(device)
    optimizer = torch.optim.Adam(sde.parameters(), lr=1e-4)
    
    losses = []

    for _ in tqdm(range(n_steps)):
        perm = torch.randperm(N_train)
        x = X_train[perm[:data_batch_size], :]
        y = y_train[perm[:data_batch_size], :]

        optimizer.zero_grad()
        
        partial_log_p = lambda params_batch: log_posterior_batch(x, y, params_batch, size_list=size_list)
#         import pdb; pdb.set_trace()
        loss = relative_entropy_control_cost(sde, partial_log_p, param_batch_size=param_batch_size, dt=dt, device=device)
        loss.backward()
        
        losses.append(loss.detach().cpu().numpy())
        optimizer.step()
        
        if stl: # double check theres no references left
            sde.drift_network_detatched.load_state_dict((sde.drift_network.state_dict()))
    
    losses = np.array(losses)
    
    return sde, losses

In [5]:



def predict(param_samples, x, y, size_list=None):
    with torch.no_grad():
        predict_func = lambda params : func_model(functional.get_params_from_array(params, size_list), x)
        predict_func = functorch.vmap(predict_func)

        preds = predict_func(param_samples)

        std, mean = torch.std_mean(preds, dim=0)
        mse = torch.mean((y_test - mean)**2)
        logp = torch.mean(log_likelihood_batch(x, y, param_samples, size_list=size_list))
        
    return std, mean, logp, mse


sigma2 = 1
sigma_n = 0.2



In [6]:
class MCFollmerDrift:
    
    def __init__(self, log_posterior, X,y, dim, device, n_samp=300, gamma=torch.tensor(1), debug=False, size_list=None):
        self.log_posterior = log_posterior
        self.debug = debug
        self.size_list=size_list
        self.log_posterior = log_posterior
        self.device = device
        self.X = X
        self.dim = dim
        self.y = y
        self.gamma = gamma
        self.n_samp = n_samp
        self.distrib = torch.distributions.multivariate_normal.MultivariateNormal(
            loc=torch.zeros(dim),
            covariance_matrix=torch.eye(dim) * torch.sqrt(gamma)
        )
        
    def g(self, thet):
        func = lambda params: self.log_posterior(self.X, self.y, params, size_list=self.size_list)
        func = functorch.vmap(func)
        lp = func(thet)
        reg = 0.5 * (thet**2).sum(dim=-1) / self.gamma
        
#             if torch.any(torch.isinf(torch.exp(lp + reg))):

        out = torch.exp(lp + reg)
        isnan = torch.isinf(torch.abs(out)) | torch.isnan(out)
        if self.debug and torch.any(isnan):
            import pdb; pdb.set_trace()
#         import pdb; pdb.set_trace()
        return out # nans exp(reg)

    def ln_g(self, thet):
        func = lambda params: self.log_posterior(self.X, self.y, params, size_list=self.size_list)
        func = functorch.vmap(func)
        lp = func(thet)
        reg = 0.5 * (thet**2).sum(dim=-1) / self.gamma
        
        out = lp + reg
        isnan = torch.isinf(torch.abs(out)) | torch.isnan(out)
        if self.debug and torch.any(isnan):
            import pdb; pdb.set_trace()
            
        return out # nans exp(reg)
        
    def mc_follmer_drift_(self, t, params, Z):
        # Using Stein Estimator for SFS drift

        g_YZt = self.g(params[None, ...] + torch.sqrt(1-t) * Z)
        num = (Z * g_YZt[..., None]).mean(dim=0)
        denom = torch.sqrt(1-t) * (g_YZt).mean(dim=0)
        
        out = num / denom[...,None]
        
        isnan = torch.isinf(torch.abs(out)) | torch.isnan(out)
        
        return out
    
    def mc_follmer_drift_stable(self, t, params, Z):
        # Using Stein Estimator for SFS drift
        N, d = Z.shape
        lnN = torch.log(torch.tensor(N)).to(self.device)
        
        ln_g_YZt = self.ln_g(params[None, ...] + torch.sqrt(1-t) * Z)
        
        Z_plus = torch.nn.functional.relu(Z)
        Z_minus = torch.nn.functional.relu(-Z)        
        
        ln_num_plus = torch.logsumexp(
            (torch.log(Z_plus) + ln_g_YZt[..., None]) - lnN,
            dim=0,
        )
        ln_num_minus = torch.logsumexp(
            (torch.log(Z_minus) + ln_g_YZt[..., None]) - lnN,
            dim=0
        )
        
        ln_denom = torch.logsumexp(
            torch.log(torch.sqrt(1-t))  + (ln_g_YZt) - lnN,
            dim=0
        )
        
        out =  torch.exp(ln_num_plus-ln_denom) - torch.exp(ln_num_minus-ln_denom)
        
        
        isnan = torch.isinf(torch.abs(out)) | torch.isnan(out)
        
        return out 
    
    def mc_follmer_drift_debug(self, t, params):
        # Using Stein Estimator for SFS drift
        
        
        Z = self.distrib.rsample((self.n_samp,)).to(self.device)
        params = params[0]

        g_YZt = self.g(params[None, ...] + torch.sqrt(1-t) * Z)
        num = (Z * g_YZt[..., None]).mean(dim=0)
        denom = torch.sqrt(1-t) * (g_YZt).mean(dim=0)
        
        out = num / denom[...,None]
        
        isnan = torch.isinf(torch.abs(out)) | torch.isnan(out)
        
        if self.debug and torch.any(isnan):
            import pdb; pdb.set_trace()
        
        return out.reshape(1,-1)
    
    def mc_follmer_drift(self, t , params_batch):
        Z = self.distrib.rsample((params_batch.shape[0], self.n_samp)).to(self.device)
        
        func = lambda params, z: self.mc_follmer_drift_stable(t, params, z)
        func = functorch.vmap(func, in_dims=(0,0) )
        out = func(params_batch, Z)
#         import pdb; pdb.set_trace()
        return out

    

class MCFollmerSDE(torch.nn.Module):

    def __init__(self, gamma, dim, log_posterior, X_train, y_train, device, debug=False, size_list=None):
        super().__init__()

        self.noise_type = 'diagonal'
        self.sde_type = 'ito'
        self.gamma = gamma
        self.size_list = size_list
        if debug:
            self.drift =  MCFollmerDrift(log_posterior, X_train, y_train, dim, device, gamma=gamma, debug=debug, size_list=size_list).mc_follmer_drift_debug
        else:
            self.drift =  MCFollmerDrift(log_posterior, X_train, y_train, dim, device, gamma=gamma, size_list=size_list).mc_follmer_drift
        self.dim = dim
        
    def f(self, t, y, detach=False):
        return self.drift(t, y)
        
    def g(self, t, y):
        return torch.sqrt(self.gamma )* torch.ones_like(y)

    def sample_trajectory(self, batch_size, dt=0.05, device=None):
        param_init = torch.zeros((batch_size, self.dim), device=device)

        n_steps = int(1.0 / dt)

        ts = torch.linspace(0, 1, n_steps, device=device)

        param_trajectory = torchsde.sdeint(self, param_init, ts, method="euler", dt=dt)

        return param_trajectory, ts

    def sample(self, batch_size, dt=0.05, device=None):
        return self.sample_trajectory(batch_size, dt=dt, device=device)[0] [-1]#[-1]
    

# mcfol = MCFollmerDrift(log_posterior, X_train, y_train, dim, device)
# sde_sfs = MCFollmerSDE(torch.tensor(gamma), dim, log_posterior, X_train, y_train, device)

In [7]:
def pred_true_std(X_train, X_test, sigma_n, sigma2, dim):
    # https://github.com/probml/pml-book/releases/latest/download/book1.pdf
    # See Eq 11.124 in the above link page 430 on pdf viewer (page 400 on page number in pdf)

    X_trainnp = X_train.cpu().detach().numpy()
    n_, d = X_trainnp.shape

    X_trainnp = np.concatenate((X_trainnp, np.ones((n_, 1))), axis=1)

    X_testnp = X_test.cpu().detach().numpy()
    n_, d = X_testnp.shape

    X_testnp = np.concatenate((X_testnp, np.ones((n_, 1))), axis=1)


    Sigma_post = sigma_n**2 * np.linalg.inv(sigma_n**2 *  np.eye(dim) / sigma2 + np.dot(X_trainnp.T,X_trainnp))
    
    sigma_pred = []
    for i in range(n_):
        sigma_pred += [np.dot(X_testnp[i,:].dot(Sigma_post), X_testnp[i,:]) + sigma_n**2 ]

    std_true = np.sqrt(sigma_pred)
    return std_true


def pred_true_mean(y_train, X_train, X_test, sigma_n, sigma2, dim):
    # https://github.com/probml/pml-book/releases/latest/download/book1.pdf
    # See Eq 11.124 in the above link page 430 on pdf viewer (page 400 on page number in pdf)

    X_trainnp = X_train.cpu().detach().numpy()
    n_, d = X_trainnp.shape
    
    lambda_ = sigma_n**2 / sigma2 

    X_trainnp = np.concatenate((X_trainnp, np.ones((n_, 1))), axis=1)

    X_testnp = X_test.cpu().detach().numpy()
    n_, d = X_testnp.shape

    X_testnp = np.concatenate((X_testnp, np.ones((n_, 1))), axis=1)

    Xty = np.dot(X_trainnp.T, y_train)

    Sigma_post = np.linalg.inv(sigma_n**2 *  np.eye(dim) / sigma2 + np.dot(X_trainnp.T,X_trainnp))
    
    w = np.dot(Sigma_post, Xty)
    
    return np.dot(X_testnp,w)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"



def lin_reg_data_gen(dim, sigma_n, device, num_samps=30):

    w = np.ones((dim,1))
    b = 1

    func = lambda x: np.dot(x, w)  + 1.0

    # Test inputs
    num_test_samples = 30

    if dim == 1:
        X_test = np.linspace(-16, 16, num_samps).reshape(num_samps,1)
        X_train = np.linspace(-3.5, 3.5, num_samps).reshape(-1,1)
    else:
        X_test  = np.random.randn(num_samps, dim)
        X_train  = np.random.randn(num_samps, dim)

    # Noise free training inputs

    #f_train = np.cos(X_train) 
    f_train = func(X_train)

    # Noise-free training outputs
    #f = np.cos(X_test)
    f = func(X_test)
    y_test  = f

    # Noisy training Inputs with additive Gaussian noise (zero-mean, variance sigma_n)

    mu = np.zeros(X_train.shape[0])
    
#     import pdb; pdb.set_trace()
    epsilon = np.random.multivariate_normal(mu, sigma_n**2 * np.eye(X_train.shape[0]))

    # Noisy targets
    try:
        y_train = f_train + epsilon.reshape(X_train.shape[0],1)
    except:
        import pdb; pdb.set_trace()
    
    return X_train, y_train, X_test, y_test, f


gamma = 0.1**2

n_steps = 300
data_batch_size = 50
param_batch_size = 32
samps_lost = []

errors = []

samp_no = 100


def log_prior(params):
    return -torch.sum(params**2) / (2 * sigma2)

def log_likelihood(x, y, params, size_list=None):
#     import pdb; pdb.set_trace()

    preds = func_model(functional.get_params_from_array(params, size_list), x)
    
    diff = preds - y
    
    return - torch.sum(diff**2) / (2 * sigma_n**2)

def log_likelihood_batch(x, y, params_batch, size_list=None):
    func = lambda params: log_likelihood(x, y, params, size_list=size_list)
    func = functorch.vmap(func)
    return func(params_batch)

def log_posterior(x, y, params, size_list=None):
    return log_prior(params) + (N_train / x.shape[0]) * log_likelihood(x, y, params, size_list=size_list)

def log_posterior_batch(x, y, params_batch, size_list=None):
    func = lambda params: log_posterior(x, y, params, size_list=size_list)
    func = functorch.vmap(func)
    return func(params_batch)

for i in range(1,11):
    dim = dim_data =  2**i


    X_train, y_train, X_test, y_test, f = lin_reg_data_gen(dim,  sigma_n, device)
    print(X_train.shape)

    N_train , _ = X_train.shape
    N_test , _ = X_test.shape
    
    X_train = torch.tensor(X_train, device=device, dtype=torch.float)
    X_test = torch.tensor(X_test, device=device, dtype=torch.float)

    y_train = torch.tensor(y_train, device=device, dtype=torch.float)
    y_test = torch.tensor(y_test, device=device, dtype=torch.float)
    
    ################################  N-SFS #########################################################
    model = LinModel(input_dim=dim).to(device)
    func_model, params = functorch.make_functional(model)
    size_list = functional.params_to_size_tuples(params)
    dim_mod = functional.get_number_of_params(size_list)
#     print(size_list)
#     
#     import pdb; pdb.set_trace()
    
    sde, losses = train(X_train, y_train, gamma, dim_mod, n_steps, data_batch_size, param_batch_size, size_list=size_list)
    param_samples = sde.sample(samp_no, dt=0.01, device=device)
    std, mean, logp, mse = predict(param_samples, X_test, y_test, size_list=size_list)
    std = torch.sqrt(std**2 + sigma_n**2)
    
    ################################  SFS ############################################################
    sde_sfs = MCFollmerSDE(torch.tensor(gamma), dim_mod, log_posterior, X_train, y_train, device, size_list=size_list)
    sfs_samps = sde_sfs.sample(samp_no, dt=0.01, device=device)
    sfs_samps = sfs_samps[~torch.isnan(sfs_samps).sum(dim=1).bool()]
    std_sfs, mean_sfs, logp_sfs, mse_sfs = predict(sfs_samps, X_test, y_test, size_list=size_list)
    std_sfs = torch.sqrt(std_sfs**2 + sigma_n**2)
    
    samps_lost.append(((~torch.isnan(sfs_samps).sum(dim=1).bool()).sum(), samp_no) )
    
    
    ################################  True ############################################################
    std_true = pred_true_std(X_train, X_test, sigma_n,  sigma2, dim_mod)
    mean_true = pred_true_mean(y_train.detach().cpu(), X_train, X_test, sigma_n,  sigma2, dim_mod)
    
    
    ################################  Errors ############################################################
    
    mae =  lambda x,y: torch.abs(torch.tensor(x).to(device)-torch.tensor(y).to(device)).sum(dim=-1).mean()
    mse =  lambda x,y: torch.sqrt(((torch.tensor(x).to(device)-torch.tensor(y).to(device) )**2).sum(dim=-1).mean())
    
    errors.append(
        {
            "mae_sfs_mean": mae(mean_sfs, mean_true),
            "mae_nsfs_mean": mae(mean, mean_true),
            "mse_sfs_mean": mse(mean_sfs, mean_true),
            "mse_nsfs_mean": mse(mean, mean_true),
            "mae_sfs_std": mae(std_sfs, std_true),
            "mae_nsfs_std": mae(std, std_true),
            "mse_sfs_std": mse(std_sfs, std_true),
            "mse_nsfs_std": mse(std, std_true),
        }
    )
#     del func_model

(30, 2)


  0%|          | 0/300 [00:00<?, ?it/s]/home/fav25/.conda/envs/functorch/lib/python3.9/site-packages/torch/nn/functional.py:2282: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::batch_norm. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at  /tmp/pip-req-build-jk216yxu/functorch/csrc/BatchedFallback.cpp:106.)
  return torch.batch_norm(
100%|██████████| 300/300 [00:16<00:00, 18.68it/s]